<a href="https://colab.research.google.com/github/7ft10/JiraExporter/blob/main/DataFlows/Jira_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#! Library Import

import os 
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

if importlib.util.find_spec("dotenv") is None:	
	os.system("pip install --quiet openai python-dotenv")

In [11]:
#! pandas Config

import pandas as pd
import requests
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

pd.options.mode.use_inf_as_na = True
#pd.set_option("display.max_rows", 10)
#pd.set_option("display.expand_frame_repr", True)
#pd.set_option('display.width', 1000)
#pd.options.display.max_seq_items = 200000
#pd.options.display.max_rows = 10

In [12]:
#! Parameters

import os 
import dotenv
import importlib.util

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

dotenv.load_dotenv('.env')

Host = os.getenv('SECRETS_HOST')
Username = os.getenv('SECRETS_USERNAME')
Password = os.getenv('SECRETS_PASSWORD')
ValidProjectCategories = ["'Portfolio'"]

if Host is None:
    raise Exception("Secrets not found")

display("Host: " + Host)
display("ValidProjectCategories: " + ','.join(ValidProjectCategories))

'Host: https://jira.budgetdirect.com.au/'

"ValidProjectCategories: 'Portfolio'"

In [13]:
#! Functions

import base64
import pandas as pd
import re 
import requests
import warnings
from functools import reduce
    
def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))
    
    if len(columnsToExpand) > 0:        
        expandedCols = expandedCols[columnsToExpand]
    
    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns] 

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0" 
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame: 
    self.columns = [fnSentenceCase(c) for c in self.columns] 
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        
        response = requests.post(Host + url, headers = headers, json = defaultContents, verify=False)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes) -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = df.expand("projects", [], None, False)    
    try:
        df = df[["issuetypes"]]
    except: 
        raise Exception("No issue metadata - check the valid project categories are correct")
    df = df.explode("issuetypes")
    df = df.expand("issuetypes", [], None, False)
    df = df[["fields"]]
    df = df.expand("fields", [], None, False)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid:pd.DataFrame = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'] if "key" in valid else valid["fieldId"],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            #display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'id', "name": 'Id', "schema_type": 'number', "required": True })
    values.append({ "fieldId": 'key', "name": 'Key', "schema_type": 'string', "required": True })
    df = pd.DataFrame(values)
    df = df.drop_duplicates().sort_values("fieldId")
    return df 

def fnGetValidProjectKeys() -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    df = fnAPI(ApiCall)
    df = df.expand("projectCategory")
    if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0:
        df = df.loc[df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df


In [14]:
#! Jira Issues Capture

from IPython.display import display

ExpectedIssueTypes = [ "Task" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") ORDER BY updatedDate DESC"

fields = fnGetIssueTypeFields(ExpectedIssueTypes)

goldenDF = None
globals()['goldenDF'] = None 

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
else:
	df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
	df = df.explode("issues")
	df = df.expand("issues", [], None, False)
	df = df.drop(["expand", "self"], axis=1)
	df = df.expand("fields", [], None, False)		

	df = df.rename( columns=dict( zip ( fields.fieldId, fields.name )) )	

	df = df.convert_dtypes().infer_objects().reset_index(drop=True)
	df["Id"] = df["Id"].astype('Int64')

	goldenDF = df.copy(deep = True)
	globals()['goldenDF'] = goldenDF # make this globally available 

if 'goldenDF' not in globals() or goldenDF is None: 
	display("Base data frame not loaded") 
else:	
	display(goldenDF.dtypes)

Id                              Int64
Key                    string[python]
Issue Type                     object
Acceptance Criteria    string[python]
Risk Probability               object
                            ...      
Story Points                  Float64
Linked Issues                  object
Assignee                       object
Updated                string[python]
Status                         object
Length: 29, dtype: object

In [15]:
#! Issues

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
   display("Base data frame not loaded") 
else:
   df = goldenDF.copy(deep = True)
   if df is None or len(df.index) == 0:	
      display("No results")
   else:
      df = df.drop(["Component/s", "Attachment", "Linked Issues", "Sprint", "Fix Version/s", "Labels"], axis=1)
      df = df.expand("Issue Type", ["id", "name"])
      df = df.expand("Status", ["id", "name", "statusCategory"]).sentence_case_columns()
      df = df.expand("Status Category", ["id", "name"]) 
      df = df.expand("Project", ["id", "name", "projectCategory"]).sentence_case_columns()
      df = df.expand("Project Category", ["id", "name"])
      df = df.expand("Reporter", ["key", "displayName"]) #df = df.expand("Reporter", ["accountId", "displayName"]) 
      df = df.expand("Assignee", ["key", "displayName"]) #df = df.expand("Assignee", ["accountId", "displayName"])
      df = df.expand("Priority", ["id", "name"])
      df = df.expand("Resolution", ["id", "name"])
      #df = df.expand("Parent", ["id", "key"])     
      #df = df.expand("Account", ["id", "value"])   
      #df = df.expand("Tempo Customer", ["id", "value"])   

      df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
      df["Story Points"] = df["Story Points"].astype('float')
      
      display(df.dtypes)
      #display(df)

Id                                Int64
Key                      string[python]
Acceptance Criteria      string[python]
Risk Probability                 object
Change Is                        object
                              ...      
Assignee Display Name    string[python]
Priority Id              string[python]
Priority Name            string[python]
Resolution Id            string[python]
Resolution Name          string[python]
Length: 34, dtype: object

In [16]:
#! Components

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Component/s"]]
    df = df[df["Component/s"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Component/s")
        df = df.expand("Component/s", ["id", "name"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

        display(df.dtypes)
        #display(df)

Id                           Int64
Key                 string[python]
Component/s Id      string[python]
Component/s Name    string[python]
dtype: object

In [17]:
#! Linked Issues

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Linked Issues"]]
    df = df[df["Linked Issues"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Linked Issues")
        df = df.expand("Linked Issues", ["id", "type", "inwardIssue", "outwardIssue"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

        display(df.dtypes)
        #display(df)

Id                                      Int64
Key                            string[python]
Linked Issues Id               string[python]
Linked Issues Type                     object
Linked Issues Inward Issue             object
Linked Issues Outward Issue            object
dtype: object

In [28]:
#! Sprints

from IPython.display import display
from dateutil import parser
from pandas.core.dtypes.dtypes import DatetimeTZDtype

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Sprint"]]
    df = df[~df["Sprint"].isna()]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:		
        df = df.explode("Sprint")
        df = df.expand("Sprint")

        if "Sprint Start Date" not in df.columns.values:
            # Server instance not cloud 
            def fixSprint(s):
                try:
                    s = (s.split('[', 1)[1])[:-1]
                    res = []
                    for sub in s.split(','):
                        if '=' in sub:
                            res.append(map(str.strip, sub.split('=', 1)))
                    return dict(res)
                except: 
                    return None
            df["Sprint"] = df["Sprint"].apply (fixSprint)
            df = df.expand("Sprint")

            def fixDate(s):
                try:
                    return parser.isoparse(s)
                except:
                    return None
            df["Sprint Start Date"] = df["Sprint Start Date"].apply(fixDate)
            df["Sprint End Date"] = df["Sprint End Date"].apply(fixDate)
            df["Sprint Complete Date"] = df["Sprint Complete Date"].apply(fixDate)
            
        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

        df["Sprint Start Date"] = df["Sprint Start Date"].astype(DatetimeTZDtype("ns", "+10:00"))
        df["Sprint End Date"] = df["Sprint End Date"].astype(DatetimeTZDtype("ns", "+10:00"))
        df["Sprint Complete Date"] = df["Sprint Complete Date"].astype(DatetimeTZDtype("ns", "+10:00"))
        
        display(df.dtypes)
        display(df)


Id                                            Int64
Key                                  string[python]
Sprint Id                            string[python]
Sprint Rapid View Id                 string[python]
Sprint State                         string[python]
                                    ...            
Sprint Complete Date      datetime64[ns, UTC+10:00]
Sprint Activated Date                string[python]
Sprint Sequence                      string[python]
Sprint Goal                          string[python]
Sprint Auto Start Stop               string[python]
Length: 13, dtype: object

,Id,Key,Sprint Id,Sprint Rapid View Id,Sprint State,Sprint Name,Sprint Start Date,Sprint End Date,Sprint Complete Date,Sprint Activated Date,Sprint Sequence,Sprint Goal,Sprint Auto Start Stop
0,677319,TAP-677,6078,1443,CLOSED,TAP: Sprint 10,2023-06-06 09:16:00+10:00,2023-06-19 23:58:00+10:00,2023-06-19 11:21:26.506000+10:00,2023-06-06T09:16:57.964+10:00,6078,,false
1,677319,TAP-677,6120,1443,CLOSED,TAP: Sprint 11,2023-06-20 08:36:00+10:00,2023-07-03 06:00:00+10:00,2023-07-03 20:54:06.047000+10:00,2023-06-20T08:37:10.323+10:00,6120,,false
2,677319,TAP-677,6207,1443,CLOSED,TAP: Sprint 12,2023-07-04 09:00:00+10:00,2023-07-17 06:30:00+10:00,2023-07-16 22:32:56.267000+10:00,2023-07-04T09:06:36.094+10:00,6207,<null>,false
3,677319,TAP-677,6275,1443,ACTIVE,TAP: Sprint 13,2023-07-17 12:00:00+10:00,2023-07-31 09:00:00+10:00,NaT,2023-07-18T08:23:16.629+10:00,6275,,false
4,677349,TAP-705,6207,1443,CLOSED,TAP: Sprint 12,2023-07-04 09:00:00+10:00,2023-07-17 06:30:00+10:00,2023-07-16 22:32:56.267000+10:00,2023-07-04T09:06:36.094+10:00,6207,<null>,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,575598,PPI-729,4354,1116,CLOSED,PPI-FY22Q3-S6,2022-03-21 08:00:00+10:00,2022-04-01 18:00:00+10:00,2022-04-04 10:08:13.778000+10:00,2022-03-21T16:36:33.299+10:00,4354,1. FLOW 2.0 - Confirm release scope with Senio...,false
535,575598,PPI-729,4463,1116,CLOSED,PPI-FY22Q4-S1,2022-04-04 08:00:00+10:00,2022-04-15 18:00:00+10:00,2022-04-14 16:31:00.990000+10:00,2022-04-04T11:41:59.468+10:00,4463,BAU 1) Risk and Issue data stabilised in Power...,false
536,571441,PPI-645,4314,1116,CLOSED,PPI-FY22Q3-S4,2022-02-21 08:00:00+10:00,2022-03-04 17:00:00+10:00,2022-03-07 10:56:35.089000+10:00,2022-02-21T20:05:04.577+10:00,4314,1) Mature the Dynamic Delivery roadmap 2) Emb...,false
537,555838,PPI-419,4031,1116,CLOSED,CHDLV-FY22-S11,2021-11-22 12:00:00+10:00,2021-12-03 18:00:00+10:00,2021-12-06 08:57:33.890000+10:00,2021-11-22T11:46:01.309+10:00,4031,Primary Goal 1. Showcase Concept Phase in conf...,false


In [ ]:
#! Attachments

from IPython.display import display
from pandas.core.dtypes.dtypes import DatetimeTZDtype

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Attachment"]]
    df = df[df["Attachment"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Attachment")
        df = df.expand("Attachment", ["id", "filename", "created", "mimeType", "size", "thumbnail", "content", "author"])
        df = df.expand("Attachment Author", ["key"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
        df["Attachment Id"] = df["Attachment Id"].astype('Int64')
        df["Attachment Size"] = df["Attachment Size"].astype('float64')
        df["Attachment Created"] = df["Attachment Created"].astype(DatetimeTZDtype("ns", "+10:00"))

        display(df.dtypes)
        #display(df)

Id                                                  Int64
Key                                        string[python]
Attachment Id                                       Int64
Attachment Filename                        string[python]
Attachment Created              datetime64[ns, UTC+10:00]
Attachment Mime Type                       string[python]
Attachment Size                                   float64
Attachment Thumbnail                       string[python]
Attachment Content                         string[python]
Attachment Author Account Id               string[python]
dtype: object

In [ ]:
#! Versions

from IPython.display import display

if 'goldenDF' not in globals() or goldenDF is None: 
    display("Base data frame not loaded") 
else:
    df = goldenDF.copy(deep = True)
    df = df[["Id", "Key", "Fix Version/s"]]
    df = df[df["Fix Version/s"].map(lambda d: len(d)) > 0]

    if df is None or len(df.index) == 0:	
        display("No results")
    else:
        df = df.explode("Fix Version/s")
        df = df.expand("Fix Version/s", ["id", "name"])

        df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
        df["Fix Version/s Id"] = df["Fix Version/s Id"].astype('Int64')

        display(df.dtypes)
        #display(df)

Id                            Int64
Key                  string[python]
Fix Versions Id               Int64
Fix Versions Name    string[python]
dtype: object